# Game code
Basic Python code to create the game

In [1]:
import random

def check_grid(i,j, recur=False):
    """check cell (i,j). If marked or already visited, it won't check it. If there is a bomb, the game will end.
    If there is not a bomb, the game will continue (or end in a win). If cell's mine_count is zero, the function will
    recursively call itself to all the neighbouring cells"""
    global game
    global win
    
    if marked[i][j] or visited[i][j]:
        return
    # check if lose
    if grid[i][j] == 1:
        visited[i][j] = 1
        game = 0
        win = -1
        print()
        print(' #################')
        print(' ### YOU LOSE! ###')
        print(' #################')
        print()
        return
    #if not lose:
    visited[i][j] = 1
    if sum([sum(row) for row in visited]) == n_row*n_col-n_mines:
        game = 0
        win = 1
        print()
        print('################')
        print('### YOU WIN! ###')
        print('################')
        print()
    if mine_count[i][j]:
        return
    else:
        for cell_i, cell_j in neighbour(i,j):
            if not visited[cell_i][cell_j]:
                check_grid(cell_i, cell_j, recur=True)
                
def mark_bomb(i,j):
    """If cell (i,j) is not visited, this function will mark that cell (or unmark if already marked).
    This is used when it is known that cell (i,j) contains a bomb"""
    if not visited[i][j]:
        if not marked[i][j]:
            marked[i][j] = 1
        else:
            marked[i][j] = 0

neighbour = lambda i,j: [cell for cell in [(i+I,j+J) for I in [-1,0,1] for J in [-1,0,1]] if 0 <= cell[0] < n_row and 0 <= cell[1] < n_col and cell != (i,j)]

def start_game():
    """starts a new game"""
    global game
    global win
    global grid
    global visited
    global marked
    global mine_count
    
    game = 1
    win = 0
    grid = [[0 for j in range(n_col)] for i in range(n_row)]
    for i,j in random.sample(sum([[(i,j) for j in range(n_col)] for i in range(n_row)], []), n_mines):
        grid[i][j] = 1
    visited = [[0 for j in range(n_col)] for i in range(n_row)]
    marked = [[0 for j in range(n_col)] for i in range(n_row)]
    mine_count = [[sum([1 for x,y in neighbour(i,j) if grid[x][y]]) if not grid[i][j] else 'M' for j in range(n_col)] for i in range(n_row)]

def show_board(board = None, show = 'visited'):
    """shows the board. If called without specifying parameters, the board will look like a classic Minesweeper game.
    show='all' reveals the entire board"""
    if board == None:
        board = mine_count
    if show == 'all': #show entire board
        brd = [[board[i][j] for j in range(n_col)] for i in range(n_row)] ## add marked condition
    elif show != 'all': #show only visited cells
        brd = [[board[i][j] if visited[i][j]==1 else ' ' for j in range(n_col)] for i in range(n_row)]
        brd = [[brd[i][j] if marked[i][j] == 0 else 'X' for j in range(n_col)] for i in range(n_row)]
    print(' # | '+ ' '.join(list(map(str,range(n_col)))))
    print('-'*(2*n_col+4))
    for i in range(n_row):
        row = ' '+str(i)+' | ' + ' '.join([str(cell) for cell in brd[i]])
        print(row)
    return


# Play the game!
`start_game()` starts a new game.

`show_board()` shows the board.

You can use `check_grid(i,j)` to visit the cell (i,j).

You can use `mark_bomb(i,j)` to mark/unmark the cell (i,j).

In [2]:
n_row = 9
n_col = 9
n_mines = 10

In [3]:
start_game()
show_board()

 # | 0 1 2 3 4 5 6 7 8
----------------------
 0 |                  
 1 |                  
 2 |                  
 3 |                  
 4 |                  
 5 |                  
 6 |                  
 7 |                  
 8 |                  


In [4]:
# comment and uncomment the following 2 lines (and execute the cell) to play the game
check_grid(1,2)
# mark_bomb(3,2)

show_board()
print()
print('remaining mines:',n_mines - sum([sum(r) for r in marked]))
print('win:',win)

 # | 0 1 2 3 4 5 6 7 8
----------------------
 0 | 0 0 0 0 0 0 0 0 0
 1 | 0 0 0 0 0 0 0 0 0
 2 | 0 0 0 0 0 1 1 2 1
 3 | 0 1 1 1 0 1      
 4 | 1 2   1 0 1      
 5 |       2 1 2      
 6 |                  
 7 |                  
 8 |                  

remaining mines: 10
win: 0


# SAT solver
Here we implement a SAT solver that will optimally solve Minesweeper.

In [5]:
from itertools import combinations

def exactly(C,k):
    """Input: C list of SAT boolean variables (integers), k integer.
    Output: formula (list of clauses) to add to the model, equivalent to "exactly k"
    """
    assert len(C) >0, "Error: C must have size at least 1"
    assert isinstance(C,list), "Error: C must be  a list"
    assert len(C) >= k, "Error: len(C) < k"
    n = len(C)
    l = []
    # at least k
    combs = combinations(C,n-k+1)
    for comb in combs:
        l.append(list(comb))
    # at most k
    combs = combinations(C,k+1)
    for comb in combs:
        l.append(list(map(lambda x: -x, list(comb))))
    return l


In [6]:
from pysat.solvers import Minisat22
import copy # needed to use the "deepcopy" function

def discovered_board():
    discovered_board = [[mine_count[i][j] if visited[i][j] else None for j in range(n_col)] for i in range(n_row)]
    for i in range(n_row):
        for j in range(n_col):
            if marked[i][j]:
                discovered_board[i][j] = 'X'
    return discovered_board

def find_safe_mines(discovered_board):
    """Finds safe squares and mines given the information contained in discovered_board.
    This function does not take into account the number of remaining mines."""
    
    # deal with marked mines
    info_board = copy.deepcopy(discovered_board)
    for i in range(n_row):
        for j in range(n_col):
            if info_board[i][j] == 'X':
                for (x,y) in neighbour(i,j):
                    if isinstance(info_board[x][y],int):
                        info_board[x][y] -= 1
    
    # unvisited boundary (--> propositions)
    unvisited_boundary = []
    for i in range(n_row):
        for j in range(n_col):
            if info_board[i][j]==None:
                for (x,y) in neighbour(i,j):
                    if info_board[x][y] != None and info_board[x][y] != 'X':
                        unvisited_boundary.append((i,j))
                        break
    
    # propositions (propositional variables)
    coords_to_int = {unvisited_boundary[i]:(i+1) for i in range(len(unvisited_boundary))}
    int_to_coords = {y: x for x, y in coords_to_int.items()}
    
    # visited_boundary (--> formulas)
    visited_boundary = []
    for i in range(n_row):
        for j in range(n_col):
            if isinstance(info_board[i][j],int):
                for (x,y) in neighbour(i,j):
                    if info_board[x][y] == None:
                        visited_boundary.append((i,j))
                        break
    
    # formulas
    formulas = []
    for (i,j) in visited_boundary:
            C = [coords_to_int[(x,y)] for (x,y) in neighbour(i,j) if info_board[x][y] == None]
            k = info_board[i][j]
            formula = exactly(C, k)
            # print('adding formula:',(i,j),formula)
            formulas.append(formula)
    
    # find safe cells and mines
    safe_list = []
    mines_list = []
    for index in int_to_coords:
        
        # find safe
        with Minisat22() as model:
            for formula in formulas: model.append_formula(formula)
            model.add_clause([index]) # here we are adding "there is a mine in 'index'", if model is unsolvable it means that there is no mine in here
            if not model.solve():
                safe_list.append(int_to_coords[index])
        
        # find mine
        with Minisat22() as model:
            for formula in formulas: model.append_formula(formula)
            model.add_clause([-index])
            if not model.solve():
                mines_list.append(int_to_coords[index])
    
    return (safe_list,mines_list)

def probability_of_safe(model):
    '''The input is a model.
    The output is a list whose first element is 0 and the i-th element is the number of models that make the proposition i false.
    (Proposition i false <-> there is no bomb in cell i <-> cell i is safe)'''
    
    model.solve()
    
    truth_count = [0 for _ in range(len(model.get_model())+1)]
    tot = 0
    for mod in model.enum_models():
        tot += 1
        for i in mod:
            if i < 0:
                truth_count[-i] += 1
    
    return list(map(lambda x: x/tot, truth_count))


def p_safe_dict(discovered_board):
    """Returns a dictionary whose keys are boundary cells and whose items are the percentages of models in which the proposition
    'the cell in the key is safe' is true"""
    
    # deal with marked mines
    info_board = copy.deepcopy(discovered_board)
    for i in range(n_row):
        for j in range(n_col):
            if info_board[i][j] == 'X':
                for (x,y) in neighbour(i,j):
                    if isinstance(info_board[x][y],int):
                        info_board[x][y] -= 1
    
    # unvisited boundary (--> propositions)
    unvisited_boundary = []
    for i in range(n_row):
        for j in range(n_col):
            if info_board[i][j]==None:
                for (x,y) in neighbour(i,j):
                    if info_board[x][y] != None and info_board[x][y] != 'X':
                        unvisited_boundary.append((i,j))
                        break
    
    # propositions (propositional variables)
    coords_to_int = {unvisited_boundary[i]:(i+1) for i in range(len(unvisited_boundary))}
    int_to_coords = {y: x for x, y in coords_to_int.items()}
    
    # visited_boundary (--> formulas)
    visited_boundary = []
    for i in range(n_row):
        for j in range(n_col):
            if isinstance(info_board[i][j],int):
                for (x,y) in neighbour(i,j):
                    if info_board[x][y] == None:
                        visited_boundary.append((i,j))
                        break
     
    # formulas
    formulas = []
    for (i,j) in visited_boundary:
            C = [coords_to_int[(x,y)] for (x,y) in neighbour(i,j) if info_board[x][y] == None]
            k = info_board[i][j]
            formula = exactly(C, k)
            # print('adding formula:',(i,j),formula)
            formulas.append(formula)
    
    # define model and output dictionary
    
    with Minisat22() as model:
        for formula in formulas: model.append_formula(formula)
        p_safe = probability_of_safe(model)
        
    return {int_to_coords[i]:p_safe[i] for i in int_to_coords.keys()}

In [7]:

random.seed(28) # see model counting

def t():
    time.sleep(0.1)

import time

start_game()

show_board()
print()
print('remaining mines:',n_mines - sum([sum(r) for r in marked]))
print('win:',win)

choose_random = True
while game == 1:
    while choose_random and game==1:
        i,j = random.randint(0,8),random.randint(0,8) ## fix this: only undiscovered cells
        print('next checking(',i,',',j,') ',sep='')
        check_grid(i,j)
        t()
        if mine_count[i][j] ==0 :
            choose_random = False
        
        show_board()
        print()
        print('remaining mines:',n_mines - sum([sum(r) for r in marked]))
        print('win:',win)
        
    if game == 0:
        break
    
    print()
    print('***********************')
    
    t()
    d_board = discovered_board()
    
    safe,mines = find_safe_mines(d_board)
    print('mines',mines)
    print('safe',safe)
    
    if len(safe) == 0 and len(mines) == 0: # model counting
        
        p_safe = p_safe_dict(d_board)
        
        print()
        print('There are no sure mines nor safe cells')
        t()
        print()
        print('Model counting returns the following probability of the cell being safe:')
        print(p_safe)
        t()
        
        max_safe = 0
        for cell in p_safe:
            if p_safe[cell] > max_safe:
                safest_cells = [cell]
                max_safe = p_safe[cell]
            elif p_safe[cell] == max_safe:
                safest_cells.append(cell)
        cell = random.choice(safest_cells)
        
        print('The safest cells are:')
        print(safest_cells)
        t()
        print('We randomly choose cell:')
        print(cell)
        t()
        
        check_grid(cell[0],cell[1])
        
        # choose_random = True
        
    if len(mines) != 0:
        print('marking mines')
        t()
        for cell in mines:
            mark_bomb(cell[0],cell[1])
    
    if len(safe) != 0:
        print('checking safe cells')
        t()
        for cell in safe:
            check_grid(cell[0],cell[1])
    
    print()
    show_board()
    print()
    print('remaining mines:',n_mines - sum([sum(r) for r in marked]))
    print('win:',win)
    print('***********************')

 # | 0 1 2 3 4 5 6 7 8
----------------------
 0 |                  
 1 |                  
 2 |                  
 3 |                  
 4 |                  
 5 |                  
 6 |                  
 7 |                  
 8 |                  

remaining mines: 10
win: 0
next checking(3,2) 
 # | 0 1 2 3 4 5 6 7 8
----------------------
 0 |                  
 1 |                  
 2 |                  
 3 |     1            
 4 |                  
 5 |                  
 6 |                  
 7 |                  
 8 |                  

remaining mines: 10
win: 0
next checking(6,2) 
 # | 0 1 2 3 4 5 6 7 8
----------------------
 0 |                  
 1 |                  
 2 |           2 2 2  
 3 |     1 1 1 1 0 1  
 4 | 1 1 1 0 0 0 0 1  
 5 | 0 0 0 0 1 1 1 1  
 6 | 0 0 0 0 1        
 7 | 0 0 0 1 2        
 8 | 0 0 0 1          

remaining mines: 10
win: 0

***********************
mines [(1, 7), (2, 4), (3, 1), (6, 5), (8, 4)]
safe [(1, 4), (2, 1), (2, 2), (2, 3), (3, 0),